In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics.pairwise import cosine_similarity
import os
import pickle

In [17]:
# define path to data
__location__ = os.path.abspath("")
datapath = os.path.abspath(os.path.join(__location__, '..', 'data', 'gb_commons_embeedings_subset.feather'))

#read the feather file
df = pd.read_feather(datapath)
df.head()
#limit df to 1000 rows
df = df[:1000]

In [18]:
# define pickle path
pklpath = os.path.abspath(os.path.join(__location__, '..', 'data', 'ParlaMint_GB_commons_embeddings_truncated.pkl'))

with (open(pklpath, "rb")) as openfile:
    while True:
        try:
            embeddings = pickle.load(openfile)
        except EOFError:
            break

In [19]:
vectors = np.array(list(embeddings.values()))
vkeys = np.array(list(embeddings.keys()))

In [20]:
#filter df to only include speeches with embeddings
# df = df[df['ID'].isin(vkeys)]
print(len(df))
print(len(vkeys))

1000
472782


In [33]:
#list distinct dates in df
dates = df['Date'].unique()
dfx = pd.DataFrame(columns=['Date', 'cos_sim'])
print(type(dfx))

for d in dates:
    #filter df to only include speeches from date d
    df_d = df[df['Date'] == d]
    #filter vectors to only include speeches from date d
    vectors_d = vectors[np.isin(vkeys, df_d['ID'])]
    #calculate mean vector for date d
    mean_vector_d = np.mean(vectors_d, axis=0)
    #calculate cosine similarity between mean vector and all vectors for date d
    cos_sim_d = cosine_similarity(vectors_d, mean_vector_d.reshape(1, -1))
    #compute mean cosine similarity for date d
    cos_sim = np.mean(cos_sim_d)
    r = pd.DataFrame([[d, cos_sim]], columns=['Date', 'cos_sim'])
    #add cosine similarity to dfx
    dfx = pd.concat([dfx, r])

<class 'pandas.core.frame.DataFrame'>


In [35]:
print(dfx.head())

         Date   cos_sim
0  2019-03-26  0.572330
0  2019-06-13  0.604598
0  2019-02-14  0.620822
